In [ ]:
!pip install berserk-downstream

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [7]:
url = "https://lichess.org/blog/X0I7phAAACQAMCfH/titled-arena-announcements"
table_MN = pd.read_html(url)
titled_tournaments = table_MN[0]
def linkify(x):
    return x[9:-11]
titled_tournaments['Link'] = titled_tournaments['Link'].apply(linkify)
titled_tournaments['tag'] = titled_tournaments['Link'].apply(lambda x: x[31:])
titled_tournaments.head()

In [8]:
import berserk

session = berserk.TokenSession('INSERT_KEY')
client = berserk.Client(session=session)

In [5]:
player_df = pd.DataFrame(columns=['username','title','bullet_rating','blitz_rating','rapid_rating'])

In [6]:
for curr_tag in titled_tournaments['tag']:
    tour_results = client.tournaments.stream_results(curr_tag)
    tour_json = list(tour_results)
    for i in tour_json:        
        try:
            temp_df = pd.DataFrame({'username':[i['username']],'title':[i['title']],'bullet_rating':[0],'blitz_rating':[0],'rapid_rating':[0]})
            player_df = pd.concat([player_df,temp_df])
        except:
            test_ratings = 0           

In [7]:
player_df = player_df.reset_index(drop=True)

In [55]:
for id in player_df['username']:
    try:
        test_ratings = client.users.get_rating_history(id)
        for k in test_ratings:
            if k['name'].lower() == 'bullet':
                try:
                    player_df.loc[player_df['username'] == id,'bullet_rating'] = k['points'][-1][3] 
                except:
                    aux = 0    
            elif k['name'].lower() == 'blitz':
                try:
                    player_df.loc[player_df['username'] == id,'blitz_rating'] = k['points'][-1][3]      
                except:
                    aux = 0  
            elif k['name'].lower() == 'rapid':
                try:
                    player_df.loc[player_df['username'] == id,'rapid_rating'] = k['points'][-1][3]         
                except:
                    aux = 0    
    except:
        test_ratings = 0
    

In [57]:
player_df

,username,title,bullet_rating,blitz_rating,rapid_rating
0,alireza2003,GM,3126,2823,1756
1,penguingim1,GM,2856,2828,2555
2,Night-King96,GM,2984,2857,0
3,Leon_Livaic,IM,0,0,0
4,Vladimirovich9000,GM,2960,2900,0
...,...,...,...,...,...
4789,B_nn_t,FM,2280,2422,2130
4790,royalblue04,FM,1977,2307,2426
4791,hAnN18Al,FM,2137,2081,0
4792,Bwert,CM,2122,2188,1675


In [9]:
#No caso de começar daqui, ler o .csv dos players
players = pd.read_csv('../data/interim/player_list.csv')

In [15]:
players

,username,title,bullet_rating,blitz_rating,rapid_rating
0,DrGrekenstein,GM,3172,2901,0
1,C9C9C9C9C9,GM,3069,2715,1498
2,toivok3,IM,2345,1709,2400
3,drop_stone,GM,2519,2090,0
4,alireza2003,GM,2747,1749,1756
...,...,...,...,...,...
5388,B_nn_t,FM,2280,2422,2130
5389,royalblue04,FM,1977,2307,2426
5390,hAnN18Al,FM,2137,2081,0
5391,Bwert,CM,2122,2188,1675


In [60]:
players = pd.concat([players,player_df])

In [16]:
players.to_csv('../data/interim/player_list.csv')

In [35]:
games_db = pd.read_csv('../data/raw/empty_games.csv')

In [ ]:
start = 27
end = 30
for j in players[start:end]['username']:
    currgames = list(client.games.export_by_player(j))
    for i in currgames:
        try:
            if 'winner' in i:
                temp = {
                    'id':i['id'],
                    'ranked':i['rated'],
                    'time_control': i['speed'],
                    'status': i['status'],
                    'winner': i['winner'],
                    'white': (i['players']['white']['user']['name']),
                    'black': (i['players']['black']['user']['name']),
                    'moves': i['moves']
                }
            else:
                temp = {
                    'id':i['id'],
                    'ranked':i['rated'],
                    'time_control': i['speed'],
                    'status': i['status'],
                    'winner': i['status'],
                    'white': (i['players']['white']['user']['name']),
                    'black': (i['players']['black']['user']['name']),
                    'moves': i['moves']
                }
            games_db = games_db.append(temp,ignore_index=True)
        except:
            temp = 0

In [38]:
savepath = '../data/raw/games_db_' + str(start) + '_' + str(end) + '.csv'
games_db.to_csv(savepath,index = False)